In [1]:
import random
import nltk
import pandas as pd
nltk.download('punkt_tab')
#from nltk.tokenize import word_tokenize
from nltk.tokenize import RegexpTokenizer

[nltk_data] Downloading package punkt_tab to
[nltk_data]     /data/ephemeral/home/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


In [2]:
# config에 저장된 데이터 경로를 통해 train과 validation data를 불러옵니다.

# train data의 구조와 내용을 확인합니다.
train_df = pd.read_csv("/home/data/train.csv")
train_df.tail()

,fname,dialogue,summary,topic
12452,train_12455,#Person1#: 실례합니다. 맨체스터 출신의 그린 씨이신가요?\n#Person2...,탄 링은 흰머리와 수염으로 쉽게 인식되는 그린 씨를 만나 호텔로 데려갈 예정입니다....,누군가를 태우다
12453,train_12456,#Person1#: 이윙 씨가 우리가 컨퍼런스 센터에 오후 4시에 도착해야 한다고 ...,#Person1#과 #Person2#는 이윙 씨가 늦지 않도록 요청했기 때문에 컨퍼...,컨퍼런스 센터
12454,train_12457,#Person1#: 오늘 어떻게 도와드릴까요?\n#Person2#: 차를 빌리고 싶...,#Person2#는 #Person1#의 도움으로 5일 동안 소형 차를 빌립니다.,차 렌트
12455,train_12458,#Person1#: 오늘 좀 행복해 보이지 않아. 무슨 일 있어?\n#Person2...,#Person2#의 엄마가 일자리를 잃었다. #Person2#는 엄마가 우울해하지 ...,실직
12456,train_12459,"#Person1#: 엄마, 다음 토요일에 이 삼촌네 가족을 방문하기 위해 비행기를 ...",#Person1#은 다음 토요일에 이 삼촌네를 방문할 때 가방을 어떻게 싸야 할지 ...,짐 싸기


In [9]:

tokenizer = RegexpTokenizer(r'#\w+#|\w+')

# 랜덤 시드를 설정하여 재현성을 확보
random.seed(1)

# 구두점 리스트 (AEDA에서 사용할 구두점 기호들)
PUNCTUATIONS = ['.', ',', '!', '?', ';', ':']

# AEDA 증강 함수 (줄바꿈을 유지)
def aeda_with_newlines(text, punctuations=PUNCTUATIONS, num_punct=3, special_tokens=None):
    # 줄바꿈을 기준으로 문장 분리
    lines = text.split('\n')
    augmented_lines = []
    
    for line in lines:
        if line.strip():  # 빈 줄은 제외
            words = tokenizer.tokenize(line)
            new_words = []
            for word in words:
                if special_tokens and word in special_tokens:
                    # Special tokens은 그대로 유지
                    new_words.append(word)
                else:
                    # 일반 단어일 경우 구두점 삽입 기회를 부여
                    new_words.append(word)
                    for _ in range(num_punct):
                        if random.random() > 0.25:  # 0.25 75%, 0.5 50% 확률이 가장 최적
                            new_words.append(random.choice(punctuations))
            augmented_lines.append(' '.join(new_words))
        else:
            # 빈 줄은 그대로 유지
            augmented_lines.append('')

    # 줄바꿈을 다시 추가하여 원래 형식으로 반환
    return '\n'.join(augmented_lines)

# 예시 텍스트 (줄바꿈 포함)와 Special tokens
text = train_df['dialogue'][1]
special_tokens = ['#Person1#',     #76737
                  '#Person2#',     #70211
                  '#Person3#',     #452
                  '#Person4#',     #41
                  '#Person5#',     #5
                  '#Person6#',     #9
                  '#Person7#',     #3
                  '#SSN#',         #3
                  '#PhoneNumber#', #203
                  '#Address#',     #45
                  '#Email#',       #17
                  '#CarNumber#',   #6
                  '#CardNumber#'   #10
                  '#DateOfBirth#', #8
                  '#PassportNumber#']  #7

# AEDA 적용 (줄바꿈 유지)
augmented_text = aeda_with_newlines(text, num_punct=2, special_tokens=special_tokens)

# 결과 출력
print(f"원문:\n{text}")
print(f"\nAEDA 적용된 문장:\n{augmented_text}")


원문:
#Person1#: 안녕하세요, 파커 부인, 어떻게 지내셨나요?
#Person2#: 안녕하세요, 피터스 박사님. 잘 지냈습니다, 감사합니다. 리키와 함께 백신 접종을 위해 왔습니다.
#Person1#: 좋습니다. 백신 접종 기록을 보니, 리키는 이미 소아마비, 디프테리아, B형 간염 백신을 맞았군요. 그는 14개월이므로, 이제 A형 간염, 수두, 홍역 백신을 맞아야 합니다.
#Person2#: 풍진과 볼거리는 어떻게 되나요?
#Person1#: 지금은 이 백신들만 접종할 수 있고, 몇 주 후에 나머지를 접종할 수 있습니다.
#Person2#: 좋습니다. 박사님, 저도 디프테리아 예방접종이 필요할 것 같아요. 마지막으로 맞은 게 아마도 15년 전이었던 것 같아요!
#Person1#: 저희가 기록을 확인하고 간호사에게 부스터를 접종하도록 하겠습니다. 이제, 리키의 팔을 꽉 잡아주세요, 조금 찌릿할 수 있습니다.

AEDA 적용된 문장:
#Person1# 안녕하세요 . 파커 ? ? 부인 , 어떻게 ? 지내셨나요 . !
#Person2# 안녕하세요 , . 피터스 . 박사님 . ? 잘 ? ; 지냈습니다 ? 감사합니다 ! 리키와 ! 함께 ? ; 백신 . 접종을 ! 위해 : ; 왔습니다 ? :
#Person1# 좋습니다 ? 백신 ; . 접종 : ? 기록을 ! : 보니 : ? 리키는 . ; 이미 ! . 소아마비 ! ; 디프테리아 ? , B형 . ; 간염 ; 백신을 ; ! 맞았군요 ; . 그는 : , 14개월이므로 ; 이제 ? ; A형 ; ! 간염 . ; 수두 ! . 홍역 : 백신을 . ! 맞아야 . 합니다
#Person2# 풍진과 , 볼거리는 ; 어떻게 , 되나요 , :
#Person1# 지금은 ? ? 이 . ! 백신들만 , ! 접종할 ; ; 수 . 있고 . , 몇 ; ; 주 : ; 후에 ; ? 나머지를 ! ? 접종할 , 수 ! 있습니다 !
#Person2# 좋습니다 ? , 박사님 . 저도 , ; 디프테리아 : . 예방접종이 ! 필요할 ? ? 것 ! 같아요 . ? 마

In [10]:
aeda_train_df = []
for i in range(len(train_df)):
    text = train_df['dialogue'][i]
    text = aeda_with_newlines(text, num_punct=2, special_tokens=special_tokens)
    aeda_train_df.append(text)
    #print(text)
    #print(aeda_train[i])


In [11]:
aeda_train_df = pd.DataFrame(aeda_train_df)


In [12]:
tmp = train_df.copy()
tmp['dialogue'] = aeda_train_df
tmp

,fname,dialogue,summary,topic
0,train_0,#Person1# 안녕하세요 : . 스미스씨 . 저는 ! 호킨스 ! ! 의사입니다 ...,"스미스씨가 건강검진을 받고 있고, 호킨스 의사는 매년 건강검진을 받는 것을 권장합니...",건강검진 받기
1,train_1,"#Person1# 안녕하세요 ? : 파커 : 부인 ; ; 어떻게 ? , 지내셨나요 ...",파커 부인이 리키를 데리고 백신 접종을 하러 갔다. 피터스 박사는 기록을 확인한 후...,백신
2,train_2,#Person1# 실례합니다 ? ? 열쇠 ; 한 ; ! 묶음 ? 보셨나요 : .\n...,"#Person1#은 열쇠 한 묶음을 찾고 있고, 그것을 찾기 위해 #Person2#...",열쇠 찾기
3,train_3,"#Person1# 왜 너는 : : 여자친구가 , 있다는 : 걸 , ? 말해주지 . ...",#Person1#은 #Person2#가 여자친구가 있고 그녀와 결혼할 것이라는 사실...,여자친구가 있다
4,train_4,"#Person1# 안녕 ? ; 숙녀분들 ! 오늘 , 밤 . , 당신들은 ? ! 정말...",말릭이 니키에게 춤을 요청한다. 말릭이 발을 밟는 것을 신경 쓰지 않는다면 니키는 ...,댄스
...,...,...,...,...
12452,train_12455,"#Person1# 실례합니다 . ? 맨체스터 . 출신의 . , 그린 . . 씨이신가...",탄 링은 흰머리와 수염으로 쉽게 인식되는 그린 씨를 만나 호텔로 데려갈 예정입니다....,누군가를 태우다
12453,train_12456,"#Person1# 이윙 ? ! 씨가 ; , 우리가 : 컨퍼런스 ; 센터에 ? 오후 ...",#Person1#과 #Person2#는 이윙 씨가 늦지 않도록 요청했기 때문에 컨퍼...,컨퍼런스 센터
12454,train_12457,"#Person1# 오늘 어떻게 , , 도와드릴까요 . !\n#Person2# 차를 ...",#Person2#는 #Person1#의 도움으로 5일 동안 소형 차를 빌립니다.,차 렌트
12455,train_12458,"#Person1# 오늘 ; . 좀 , 행복해 . 보이지 : 않아 무슨 : 일 ! 있...",#Person2#의 엄마가 일자리를 잃었다. #Person2#는 엄마가 우울해하지 ...,실직


In [13]:
ex = pd.concat([train_df, tmp], axis=0)
ex

,fname,dialogue,summary,topic
0,train_0,"#Person1#: 안녕하세요, 스미스씨. 저는 호킨스 의사입니다. 오늘 왜 오셨나...","스미스씨가 건강검진을 받고 있고, 호킨스 의사는 매년 건강검진을 받는 것을 권장합니...",건강검진 받기
1,train_1,"#Person1#: 안녕하세요, 파커 부인, 어떻게 지내셨나요?\n#Person2#...",파커 부인이 리키를 데리고 백신 접종을 하러 갔다. 피터스 박사는 기록을 확인한 후...,백신
2,train_2,"#Person1#: 실례합니다, 열쇠 한 묶음 보셨나요?\n#Person2#: 어떤...","#Person1#은 열쇠 한 묶음을 찾고 있고, 그것을 찾기 위해 #Person2#...",열쇠 찾기
3,train_3,#Person1#: 왜 너는 여자친구가 있다는 걸 말해주지 않았어?\n#Person...,#Person1#은 #Person2#가 여자친구가 있고 그녀와 결혼할 것이라는 사실...,여자친구가 있다
4,train_4,"#Person1#: 안녕, 숙녀분들! 오늘 밤 당신들은 정말 멋져 보여. 이 춤을 ...",말릭이 니키에게 춤을 요청한다. 말릭이 발을 밟는 것을 신경 쓰지 않는다면 니키는 ...,댄스
...,...,...,...,...
12452,train_12455,"#Person1# 실례합니다 . ? 맨체스터 . 출신의 . , 그린 . . 씨이신가...",탄 링은 흰머리와 수염으로 쉽게 인식되는 그린 씨를 만나 호텔로 데려갈 예정입니다....,누군가를 태우다
12453,train_12456,"#Person1# 이윙 ? ! 씨가 ; , 우리가 : 컨퍼런스 ; 센터에 ? 오후 ...",#Person1#과 #Person2#는 이윙 씨가 늦지 않도록 요청했기 때문에 컨퍼...,컨퍼런스 센터
12454,train_12457,"#Person1# 오늘 어떻게 , , 도와드릴까요 . !\n#Person2# 차를 ...",#Person2#는 #Person1#의 도움으로 5일 동안 소형 차를 빌립니다.,차 렌트
12455,train_12458,"#Person1# 오늘 ; . 좀 , 행복해 . 보이지 : 않아 무슨 : 일 ! 있...",#Person2#의 엄마가 일자리를 잃었다. #Person2#는 엄마가 우울해하지 ...,실직


In [14]:
ex.to_csv("/home/data/train_2x75p.csv", index=False)  # 결과를 CSV 파일로 저장합니다.

In [15]:

tokenizer = RegexpTokenizer(r'#\w+#|\w+')

# 랜덤 시드를 설정하여 재현성을 확보
random.seed(1)

# 구두점 리스트 (AEDA에서 사용할 구두점 기호들)
PUNCTUATIONS = ['.', ',', '!', '?', ';', ':']

# AEDA 증강 함수 (줄바꿈을 유지)
def aeda_with_newlines(text, punctuations=PUNCTUATIONS, num_punct=3, special_tokens=None):
    # 줄바꿈을 기준으로 문장 분리
    lines = text.split('\n')
    augmented_lines = []
    
    for line in lines:
        if line.strip():  # 빈 줄은 제외
            words = tokenizer.tokenize(line)
            new_words = []
            for word in words:
                if special_tokens and word in special_tokens:
                    # Special tokens은 그대로 유지
                    new_words.append(word)
                else:
                    # 일반 단어일 경우 구두점 삽입 기회를 부여
                    new_words.append(word)
                    for _ in range(num_punct):
                        if random.random() > 0.5:  # 0.25 75%, 0.5 50% 확률이 가장 최적
                            new_words.append(random.choice(punctuations))
            augmented_lines.append(' '.join(new_words))
        else:
            # 빈 줄은 그대로 유지
            augmented_lines.append('')

    # 줄바꿈을 다시 추가하여 원래 형식으로 반환
    return '\n'.join(augmented_lines)

# 예시 텍스트 (줄바꿈 포함)와 Special tokens
text = train_df['dialogue'][1]
special_tokens = ['#Person1#',     #76737
                  '#Person2#',     #70211
                  '#Person3#',     #452
                  '#Person4#',     #41
                  '#Person5#',     #5
                  '#Person6#',     #9
                  '#Person7#',     #3
                  '#SSN#',         #3
                  '#PhoneNumber#', #203
                  '#Address#',     #45
                  '#Email#',       #17
                  '#CarNumber#',   #6
                  '#CardNumber#'   #10
                  '#DateOfBirth#', #8
                  '#PassportNumber#']  #7

# AEDA 적용 (줄바꿈 유지)
augmented_text = aeda_with_newlines(text, num_punct=2, special_tokens=special_tokens)

# 결과 출력
print(f"원문:\n{text}")
print(f"\nAEDA 적용된 문장:\n{augmented_text}")


원문:
#Person1#: 안녕하세요, 파커 부인, 어떻게 지내셨나요?
#Person2#: 안녕하세요, 피터스 박사님. 잘 지냈습니다, 감사합니다. 리키와 함께 백신 접종을 위해 왔습니다.
#Person1#: 좋습니다. 백신 접종 기록을 보니, 리키는 이미 소아마비, 디프테리아, B형 간염 백신을 맞았군요. 그는 14개월이므로, 이제 A형 간염, 수두, 홍역 백신을 맞아야 합니다.
#Person2#: 풍진과 볼거리는 어떻게 되나요?
#Person1#: 지금은 이 백신들만 접종할 수 있고, 몇 주 후에 나머지를 접종할 수 있습니다.
#Person2#: 좋습니다. 박사님, 저도 디프테리아 예방접종이 필요할 것 같아요. 마지막으로 맞은 게 아마도 15년 전이었던 것 같아요!
#Person1#: 저희가 기록을 확인하고 간호사에게 부스터를 접종하도록 하겠습니다. 이제, 리키의 팔을 꽉 잡아주세요, 조금 찌릿할 수 있습니다.

AEDA 적용된 문장:
#Person1# 안녕하세요 . 파커 부인 , 어떻게 지내셨나요 ? .
#Person2# 안녕하세요 ! , 피터스 . . 박사님 . 잘 ? ? 지냈습니다 ; 감사합니다 리키와 함께 백신 접종을 ; . 위해 ! 왔습니다
#Person1# 좋습니다 : ? 백신 : 접종 ? 기록을 ; 보니 ? 리키는 ? ! 이미 : : 소아마비 디프테리아 , ? B형 ? 간염 ; 백신을 ? , 맞았군요 . ; 그는 ; 14개월이므로 ; 이제 : : A형 : 간염 , ; 수두 ? 홍역 ; ; 백신을 ? 맞아야 ; 합니다 !
#Person2# 풍진과 볼거리는 . 어떻게 ! 되나요 .
#Person1# 지금은 이 , 백신들만 , 접종할 수 , : 있고 ? ? 몇 주 후에 나머지를 ; ; 접종할 , 수 있습니다 ,
#Person2# 좋습니다 ? , 박사님 : ; 저도 . 디프테리아 ! 예방접종이 ? 필요할 . 것 ! 같아요 ! ? 마지막으로 , 맞은 . , 게 ; 아마도 : . 15년 전이었던 ? 것 ? 같아요 ! .
#Person1# 저희가 ! 기록을

In [16]:
aeda_train_df = []
for i in range(len(train_df)):
    text = train_df['dialogue'][i]
    text = aeda_with_newlines(text, num_punct=2, special_tokens=special_tokens)
    aeda_train_df.append(text)
    #print(text)
    #print(aeda_train[i])


In [17]:
aeda_train_df = pd.DataFrame(aeda_train_df)


In [18]:
tmp = train_df.copy()
tmp['dialogue'] = aeda_train_df
tmp

,fname,dialogue,summary,topic
0,train_0,#Person1# 안녕하세요 ? 스미스씨 ; 저는 ; 호킨스 . 의사입니다 ! 오늘...,"스미스씨가 건강검진을 받고 있고, 호킨스 의사는 매년 건강검진을 받는 것을 권장합니...",건강검진 받기
1,train_1,#Person1# 안녕하세요 ! ? 파커 ; 부인 : 어떻게 지내셨나요 ; :\n#...,파커 부인이 리키를 데리고 백신 접종을 하러 갔다. 피터스 박사는 기록을 확인한 후...,백신
2,train_2,"#Person1# 실례합니다 : ; 열쇠 ; 한 ! , 묶음 . 보셨나요\n#Per...","#Person1#은 열쇠 한 묶음을 찾고 있고, 그것을 찾기 위해 #Person2#...",열쇠 찾기
3,train_3,#Person1# 왜 : 너는 여자친구가 있다는 걸 : ? 말해주지 ? 않았어\n#...,#Person1#은 #Person2#가 여자친구가 있고 그녀와 결혼할 것이라는 사실...,여자친구가 있다
4,train_4,#Person1# 안녕 ! 숙녀분들 오늘 : 밤 . ! 당신들은 . 정말 : 멋져 ...,말릭이 니키에게 춤을 요청한다. 말릭이 발을 밟는 것을 신경 쓰지 않는다면 니키는 ...,댄스
...,...,...,...,...
12452,train_12455,#Person1# 실례합니다 ; 맨체스터 출신의 ; 그린 씨이신가요 !\n#Pers...,탄 링은 흰머리와 수염으로 쉽게 인식되는 그린 씨를 만나 호텔로 데려갈 예정입니다....,누군가를 태우다
12453,train_12456,"#Person1# 이윙 : ! 씨가 ! 우리가 , 컨퍼런스 , . 센터에 ; 오후 ...",#Person1#과 #Person2#는 이윙 씨가 늦지 않도록 요청했기 때문에 컨퍼...,컨퍼런스 센터
12454,train_12457,#Person1# 오늘 ; : 어떻게 ; 도와드릴까요 .\n#Person2# 차를 ...,#Person2#는 #Person1#의 도움으로 5일 동안 소형 차를 빌립니다.,차 렌트
12455,train_12458,"#Person1# 오늘 ? 좀 행복해 : 보이지 , : 않아 ! , 무슨 ; 일 !...",#Person2#의 엄마가 일자리를 잃었다. #Person2#는 엄마가 우울해하지 ...,실직


In [19]:
ex = pd.concat([ex, tmp], axis=0)
ex

,fname,dialogue,summary,topic
0,train_0,"#Person1#: 안녕하세요, 스미스씨. 저는 호킨스 의사입니다. 오늘 왜 오셨나...","스미스씨가 건강검진을 받고 있고, 호킨스 의사는 매년 건강검진을 받는 것을 권장합니...",건강검진 받기
1,train_1,"#Person1#: 안녕하세요, 파커 부인, 어떻게 지내셨나요?\n#Person2#...",파커 부인이 리키를 데리고 백신 접종을 하러 갔다. 피터스 박사는 기록을 확인한 후...,백신
2,train_2,"#Person1#: 실례합니다, 열쇠 한 묶음 보셨나요?\n#Person2#: 어떤...","#Person1#은 열쇠 한 묶음을 찾고 있고, 그것을 찾기 위해 #Person2#...",열쇠 찾기
3,train_3,#Person1#: 왜 너는 여자친구가 있다는 걸 말해주지 않았어?\n#Person...,#Person1#은 #Person2#가 여자친구가 있고 그녀와 결혼할 것이라는 사실...,여자친구가 있다
4,train_4,"#Person1#: 안녕, 숙녀분들! 오늘 밤 당신들은 정말 멋져 보여. 이 춤을 ...",말릭이 니키에게 춤을 요청한다. 말릭이 발을 밟는 것을 신경 쓰지 않는다면 니키는 ...,댄스
...,...,...,...,...
12452,train_12455,#Person1# 실례합니다 ; 맨체스터 출신의 ; 그린 씨이신가요 !\n#Pers...,탄 링은 흰머리와 수염으로 쉽게 인식되는 그린 씨를 만나 호텔로 데려갈 예정입니다....,누군가를 태우다
12453,train_12456,"#Person1# 이윙 : ! 씨가 ! 우리가 , 컨퍼런스 , . 센터에 ; 오후 ...",#Person1#과 #Person2#는 이윙 씨가 늦지 않도록 요청했기 때문에 컨퍼...,컨퍼런스 센터
12454,train_12457,#Person1# 오늘 ; : 어떻게 ; 도와드릴까요 .\n#Person2# 차를 ...,#Person2#는 #Person1#의 도움으로 5일 동안 소형 차를 빌립니다.,차 렌트
12455,train_12458,"#Person1# 오늘 ? 좀 행복해 : 보이지 , : 않아 ! , 무슨 ; 일 !...",#Person2#의 엄마가 일자리를 잃었다. #Person2#는 엄마가 우울해하지 ...,실직


In [20]:
ex.to_csv("/home/data/train_3x5075p.csv", index=False)  # 결과를 CSV 파일로 저장합니다.

## 기타

In [ ]:
#!pip install koeda


In [ ]:
### 깃허브 예시

from koeda import EDA

eda = EDA(morpheme_analyzer="Okt", alpha_sr=0.3, alpha_ri=0.3, alpha_rs=0.3, prob_rd=0.3)

text = "아버지가 방에 들어가신다"

result = eda(text)
print(result)
# 아버지가 정실에 들어가신다

result = eda(text, p=(0.9, 0.9, 0.9, 0.9), repetition=2)
print(result)
# ['아버지가 객실 아빠 안방 방에 정실 들어가신다', '아버지가 탈의실 방 휴게실 에 안방 탈의실 들어가신다']

In [ ]:
### 깃허브 예시

from koeda import AEDA

aeda = AEDA(morpheme_analyzer="Okt", punc_ratio=0.3, punctuations=[".", ",", "!", "?", ";", ":"])

text = "어머니가 집을 나가신다"

result = aeda(text)
print(result)
# 어머니가 ! 집을 , 나가신다

result = aeda(text, p=0.9, repetition=2)
print(result)
# ['! 어머니가 ! 집 ; 을 ? 나가신다', '. 어머니 ? 가 . 집 , 을 , 나가신다']

In [ ]:
#for i in range(len(train_df)):
print(val_df['dialogue'][1])